# Adapt pipeline to non-esterase enzymes

In [1]:
from filtering_pipeline.utils.helpers import log_section, log_subsection, log_boxed_note, generate_boltz_structure_path
from filtering_pipeline.utils.helpers import clean_protein_sequence, delete_empty_subdirs, add_metrics_to_best_structures
from filtering_pipeline.steps.predict_catalyticsite_step import ActiveSitePred
from filtering_pipeline.steps.save_step import Save
from filtering_pipeline.steps.extract_docking_metrics_step import DockingMetrics
from filtering_pipeline.steps.preparevina_step import PrepareVina
from filtering_pipeline.steps.preparechai_step import PrepareChai
from filtering_pipeline.steps.prepareboltz_step import PrepareBoltz
from filtering_pipeline.steps.superimposestructures_step import SuperimposeStructures
from filtering_pipeline.steps.computeproteinRMSD_step import ProteinRMSD
from filtering_pipeline.steps.computeligandRMSD_step import LigandRMSD
from filtering_pipeline.steps.geometric_filtering import GeometricFiltering
from filtering_pipeline.steps.fpocket_step import Fpocket
from filtering_pipeline.steps.ligandSASA_step import LigandSASA
from filtering_pipeline.steps.plip_step import PLIP

from enzymetk.dock_chai_step import Chai
from enzymetk.dock_boltz_step import Boltz
from enzymetk.dock_vina_step import Vina


# Example
import pandas as pd

smiles_heme = r"CC1=C(/C2=C/C3=N/C(C(C)=C3CCC(O)=O)=C\C4=C(C(C=C)=C(/C=C5N=C(C(C=C)=C\5C)/C=C1\N26)N4[Fe]6=N)C)CCC(O)=O"
smiles_substrate = "C=CC1=CC=C(OC)C=C1"
entries = ["P53553", "P53554"]
sequences = [
    "MAVPGYDFGKVPDAPISDADFESLKKTVMWGEEDEKYRKMACEALKGQVEDILDLWYGLQGSNQHLIYYFGDKSGRPIPQYLEAVRKRFGLWIIDTLCKPLDRQWLNYMYEIGLRHHRTKKGKTDGVDTVEHIPLRYMIAFIAPIGLTIKPILEKSGHPPEAVERMWAAWVKLVVLQVAIWSYPYAKTGEW",
    "MTIASSTASSEFLKNPYSFYDTLRAVHPIYKGSFLKYPGWYVTGYEETAAILKDARFKVRTPLPESSTKYQDLSHVQNQMMLFQNQPDHRRLRTLASGAFTPRTTESYQPYIIETVHHLLDQVQGKKKMEVISDFAFPLASFVIANIIGVPEEDREQLKEWAASLELIQKRKRHPQQDMISMLLKGREKDKLTEEEAASTCILLAIAGHETTVNLISNSVLCLLQHPEQLLKLRENPDLIGTA"  #  VEECLRYESPTQMTARVASEDIDICGVTIRQGEQVYLLLGAANRDPSIFTNPDVFDITRSPNPHLSFGHGHHVCLGSSLARLEAQIAINTLLQRMPSLNLADFEWRYRPLFGFRALEELPVTFE"
]
df = pd.DataFrame({
    "Entry": entries,
    "Sequence": sequences
})

output_dir = "output/docking"
ligand_smiles = smiles_substrate
cofactor_smiles = smiles_heme
ligand_name = "rando"

Enabling RDKit 2024.09.6 jupyter extensions
/home/helen/miniconda3/envs/enzymetk/lib/python3.11/site-packages/pandera/_pandas_deprecated.py:157: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


### Docking

In [4]:
import sys
import os
sys.path.insert(0, '/nvme2/helen/EnzymeStructuralFiltering/')
import filtering_pipeline
print(filtering_pipeline.__file__)  

import pandas as pd
from filtering_pipeline.steps.predict_catalyticsite_step import ActiveSitePred
from filtering_pipeline.steps.save_step import Save
from filtering_pipeline.steps.step import Step

squidly = df << ActiveSitePred('Entry', 'Sequence', '/nvme2/ariane/home/data/models/squidly_final_models/', 2)

df_squidly = pd.merge(df, squidly, left_on='Entry', right_on='label', how='inner')
output_path = os.path.join(output_dir, 'squidly.pkl')
df_squidly.to_pickle(output_path)

/home/helen/miniconda3/envs/enzymetk/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


/nvme2/helen/EnzymeStructuralFiltering/filtering_pipeline/__init__.py


  0%|          | 0/2 [00:00<?, ?it/s]conda run -n AS_inference python /nvme2/ariane/home/data/models/squidly_final_models//SQUIDLY_run_model_LSTM.py               /tmp/tmpelmirlbl/as_inference_qFm6LBkEzF.fasta esm2_t36_3B_UR50D /nvme2/ariane/home/data/models/squidly_final_models//Squidly_CL_3B.pt /nvme2/ariane/home/data/models/squidly_final_models//Squidly_LSTM_3B.pth /tmp/tmpelmirlbl               --toks_per_batch 5 --AS_threshold 0.97
Transferred representation models to GPU
Transferred AS models to GPU
Read /tmp/tmpelmirlbl/as_inference_qFm6LBkEzF.fasta with 1 sequences
Processing 1 of 1 batches (1 sequences)


 50%|█████     | 1/2 [00:34<00:34, 34.78s/it]conda run -n AS_inference python /nvme2/ariane/home/data/models/squidly_final_models//SQUIDLY_run_model_LSTM.py               /tmp/tmpelmirlbl/as_inference_TmLBStK3DY.fasta esm2_t36_3B_UR50D /nvme2/ariane/home/data/models/squidly_final_models//Squidly_CL_3B.pt /nvme2/ariane/home/data/models/squidly_final_models//Squidly_LSTM_3B.pth

['/tmp/tmpelmirlbl/as_inference_qFm6LBkEzF_results.pkl', '/tmp/tmpelmirlbl/as_inference_TmLBStK3DY_results.pkl']


In [5]:
from pathlib import Path
import pandas as pd


df_squidly = pd.read_pickle(Path(output_dir) / 'squidly.pkl')
chai_dir = Path(output_dir) / 'chai'
chai_dir.mkdir(exist_ok=True, parents=True)
df_squidly.loc[:, 'substrate'] = ligand_smiles
df_squidly.loc[:, 'cofactor'] = cofactor_smiles
df_chai = df_squidly << (Chai('Entry', 'Sequence', 'substrate', 'cofactor', chai_dir, 1) >> Save(Path(output_dir)/'chai.pkl'))
df_chai.rename(columns = {'output_dir':'chai_dir'}, inplace=True)

[fasta] [output/docking/chai/P53553/P53553.fasta] protein|P53553 191
[fasta] [output/docking/chai/P53553/P53553.fasta] ligand|P53553-cofactor 103
[fasta] [output/docking/chai/P53553/P53553.fasta] ligand|P53553-substrate-0 18


P53553 MAVPGYDFGKVPDAPISDADFESLKKTVMWGEEDEKYRKMACEALKGQVEDILDLWYGLQGSNQHLIYYFGDKSGRPIPQYLEAVRKRFGLWIIDTLCKPLDRQWLNYMYEIGLRHHRTKKGKTDGVDTVEHIPLRYMIAFIAPIGLTIKPILEKSGHPPEAVERMWAAWVKLVVLQVAIWSYPYAKTGEW C=CC1=CC=C(OC)C=C1
C=CC1=CC=C(OC)C=C1
output/docking/chai/P53553


Generating ref conformer for LIG, CC1=C(/C2=C/C3=N/C(C(C)=C3CCC(O)=O)=C\C4=C(C(C=C)=C(/C=C5N=C(C(C=C)=C\5C)/C=C1\N26)N4[Fe]6=N)C)CCC(O)=O
[20:59:37] UFFTYPER: Unrecognized atom type: Fe5+2 (36)
Generating ref conformer for LIG, C=CC1=CC=C(OC)C=C1
Trunk sample 1/1
Diffusion steps: 100%|██████████| 199/199 [00:36<00:00,  5.41it/s]


Score=0.6835, writing output to output/docking/chai/P53553/chai/pred.model_idx_0.cif


saved cif file to output/docking/chai/P53553/chai/pred.model_idx_0.cif


...

saved cif file to output/docking/chai/P53554/chai/pred.model_idx_2.cif


Score=0.3433, writing output to output/docking/chai/P53554/chai/pred.model_idx_3.cif


saved cif file to output/docking/chai/P53554/chai/pred.model_idx_3.cif


Score=0.3406, writing output to output/docking/chai/P53554/chai/pred.model_idx_4.cif


saved cif file to output/docking/chai/P53554/chai/pred.model_idx_4.cif


In [6]:
boltz_dir = Path(output_dir) / 'boltz/'
boltz_dir.mkdir(exist_ok=True, parents=True)
df_chai['cofactor'] = cofactor_smiles
df_boltz = df_chai << (Boltz('Entry', 'Sequence', 'substrate', 'cofactor', boltz_dir, 1) 
                    >> Save(Path(output_dir)/'boltz.pkl'))
df_boltz.rename(columns = {'output_dir':'boltz_dir'}, inplace=True)

P53553 MAVPGYDFGKVPDAPISDADFESLKKTVMWGEEDEKYRKMACEALKGQVEDILDLWYGLQGSNQHLIYYFGDKSGRPIPQYLEAVRKRFGLWIIDTLCKPLDRQWLNYMYEIGLRHHRTKKGKTDGVDTVEHIPLRYMIAFIAPIGLTIKPILEKSGHPPEAVERMWAAWVKLVVLQVAIWSYPYAKTGEW C=CC1=CC=C(OC)C=C1
output/docking/boltz/P53553
MSA server enabled: https://api.colabfold.com
MSA server authentication: no credentials provided
Checking input data.
Processing 1 inputs with 1 threads.


  0%|          | 0/1 [00:00<?, ?it/s]

Generating MSA for output/docking/boltz/P53553/P53553.yaml with 1 protein entities.
Calling MSA server for target P53553 with 1 sequences
MSA server URL: https://api.colabfold.com
MSA pairing strategy: greedy
No authentication provided for MSA server



100%|██████████| 1/1 [00:03<00:00,  3.49s/it]
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/helen/miniconda3/envs/enzymetk/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


Running structure prediction for 1 input.


...

Running structure prediction for 1 input.


/home/helen/miniconda3/envs/enzymetk/lib/python3.11/site-packages/pytorch_lightning/utilities/migration/utils.py:56: The loaded checkpoint was produced with Lightning v2.5.0.post0, which is newer than your current Lightning version: v2.5.0
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:31<00:00,  0.03it/s]

Predicting property: affinity

Checking input data for affinity.
Running affinity prediction for 1 input.


/home/helen/miniconda3/envs/enzymetk/lib/python3.11/site-packages/pytorch_lightning/utilities/migration/utils.py:56: The loaded checkpoint was produced with Lightning v2.5.0.post0, which is newer than your current Lightning version: v2.5.0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:23<00:00,  0.04it/s]


In [14]:
vina_dir = Path(output_dir) / 'vina/' 
vina_dir.mkdir(exist_ok=True, parents=True)
delete_empty_subdirs(vina_dir)

df_boltz = pd.read_pickle('output/docking/boltz.pkl')
df_boltz = df_boltz.rename(columns={'output_dir': 'boltz_dir'})

df_boltz['substrate_name'] = ligand_name
metagenomic_enzymes = 1

if metagenomic_enzymes == 1:
            log_boxed_note('Using structures generated by Boltz2 as input for Vina docking')
            df_boltz['structure'] = df_boltz['boltz_dir'].apply(generate_boltz_structure_path)
else: 
    df_boltz['structure'] = None # or path to AF structure
        

df_boltz['Squidly_CR_Position'] = '60|88' #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Initial Vina docking attempt
df_vina = df_boltz << (Vina('Entry', 'structure', 'Sequence', 'substrate', 'substrate_name', 'Squidly_CR_Position', vina_dir, 1))
df_vina.rename(columns = {'output_dir':'vina_dir'}, inplace=True)
df_vina.to_pickle(Path(output_dir)/'vina.pkl') 


----------------------------------------------------------------------
|   Using structures generated by Boltz2 as input for Vina docking   |
----------------------------------------------------------------------



output/docking/boltz/P53553/boltz_results_P53553/predictions/P53553/P53553_model_0.cif
output/docking/boltz/P53554/boltz_results_P53554/predictions/P53554/P53554_model_0.cif
Expects active site residues as a string separated by |. Zero indexed.
output/docking/vina/P53553/P53553.pdb


1 molecule converted


<Residue GLY het=  resseq=61 icode= >
<Residue PHE het=  resseq=89 icode= >
[array([ -5.454, -15.667,  -1.297], dtype=float32), array([-5.76 , -1.258,  3.386], dtype=float32)]
vina --config output/docking/vina/P53553/P53553-rando_conf.txt --out output/docking/vina/P53553/P53553-rando.pdb
output/docking/vina/P53554/P53554.pdb


1 molecule converted


<Residue THR het=  resseq=61 icode= >
<Residue HIS het=  resseq=89 icode= >
[array([ -1.668,  13.5  , -11.057], dtype=float32), array([  0.856,   1.711, -17.444], dtype=float32)]
vina --config output/docking/vina/P53554/P53554-rando_conf.txt --out output/docking/vina/P53554/P53554-rando.pdb


In [15]:
df = pd.read_pickle('output/docking/vina.pkl')
df_metrics = df << (DockingMetrics(input_dir = Path(output_dir), output_dir = Path(output_dir)) 
                        >> Save(Path(output_dir) / 'dockingmetrics.pkl'))

### Superimposition

In [16]:
from pathlib import Path
import pandas as pd

output_dir = 'output/superimposition'
df_metrics = pd.read_pickle('output/docking/dockingmetrics.pkl')
preparedfiles_dir = Path(output_dir) / 'preparedfiles_for_superimposition/'
df_prep = df_metrics << (PrepareVina('vina_dir', ligand_name,  preparedfiles_dir) 
        >> PrepareChai('chai_dir', preparedfiles_dir, heme=1, num_threads=1) 
        >> PrepareBoltz('boltz_dir' , preparedfiles_dir, 1))

output_sup_dir = Path(output_dir) / 'superimposed_structures'
df_sup = df_prep << (SuperimposeStructures('vina_files_for_superimposition',  'chai_files_for_superimposition',  output_dir = output_sup_dir, name1='vina', name2='chai', num_threads = 1) 
        >> SuperimposeStructures('vina_files_for_superimposition',  'boltz_files_for_superimposition',  output_dir = output_sup_dir, name1='vina', name2='boltz', num_threads = 1) 
        >> SuperimposeStructures('chai_files_for_superimposition',  'boltz_files_for_superimposition',  output_dir = output_sup_dir, name1='chai', name2='boltz', num_threads = 1)
        >> Save(Path(output_dir) / 'superimposedstructures.pkl'))

Superimposing structures: 100%|██████████| 2/2 [00:05<00:00,  2.68s/it]


In [1]:
from filtering_pipeline.utils.helpers import log_section, log_subsection, log_boxed_note, generate_boltz_structure_path
from filtering_pipeline.utils.helpers import clean_protein_sequence, delete_empty_subdirs, add_metrics_to_best_structures
from filtering_pipeline.steps.predict_catalyticsite_step import ActiveSitePred
from filtering_pipeline.steps.save_step import Save
from filtering_pipeline.steps.extract_docking_metrics_step import DockingMetrics
from filtering_pipeline.steps.preparevina_step import PrepareVina
from filtering_pipeline.steps.preparechai_step import PrepareChai
from filtering_pipeline.steps.prepareboltz_step import PrepareBoltz
from filtering_pipeline.steps.superimposestructures_step import SuperimposeStructures
from filtering_pipeline.steps.computeproteinRMSD_step import ProteinRMSD
from filtering_pipeline.steps.computeligandRMSD_step import LigandRMSD
from filtering_pipeline.steps.geometric_filtering import GeometricFiltering
from filtering_pipeline.steps.fpocket_step import Fpocket
from filtering_pipeline.steps.ligandSASA_step import LigandSASA
from filtering_pipeline.steps.plip_step import PLIP

from enzymetk.dock_chai_step import Chai
from enzymetk.dock_boltz_step import Boltz
from enzymetk.dock_vina_step import Vina

from pathlib import Path


# Example
import pandas as pd

smiles_heme = r"CC1=C(/C2=C/C3=N/C(C(C)=C3CCC(O)=O)=C\C4=C(C(C=C)=C(/C=C5N=C(C(C=C)=C\5C)/C=C1\N26)N4[Fe]6=N)C)CCC(O)=O"
smiles_substrate = "C=CC1=CC=C(OC)C=C1"   
entries = ["P53553", "P53554"]
sequences = [
    "MAVPGYDFGKVPDAPISDADFESLKKTVMWGEEDEKYRKMACEALKGQVEDILDLWYGLQGSNQHLIYYFGDKSGRPIPQYLEAVRKRFGLWIIDTLCKPLDRQWLNYMYEIGLRHHRTKKGKTDGVDTVEHIPLRYMIAFIAPIGLTIKPILEKSGHPPEAVERMWAAWVKLVVLQVAIWSYPYAKTGEW",
    "MTIASSTASSEFLKNPYSFYDTLRAVHPIYKGSFLKYPGWYVTGYEETAAILKDARFKVRTPLPESSTKYQDLSHVQNQMMLFQNQPDHRRLRTLASGAFTPRTTESYQPYIIETVHHLLDQVQGKKKMEVISDFAFPLASFVIANIIGVPEEDREQLKEWAASLELIQKRKRHPQQDMISMLLKGREKDKLTEEEAASTCILLAIAGHETTVNLISNSVLCLLQHPEQLLKLRENPDLIGTA"  #  VEECLRYESPTQMTARVASEDIDICGVTIRQGEQVYLLLGAANRDPSIFTNPDVFDITRSPNPHLSFGHGHHVCLGSSLARLEAQIAINTLLQRMPSLNLADFEWRYRPLFGFRALEELPVTFE"
]
df = pd.DataFrame({
    "Entry": entries,
    "Sequence": sequences
})

output_dir = "output/docking"
ligand_smiles = smiles_substrate
cofactor_smiles = smiles_heme
ligand_name = "rando"

Enabling RDKit 2024.09.6 jupyter extensions
/home/helen/miniconda3/envs/enzymetk/lib/python3.11/site-packages/pandera/_pandas_deprecated.py:157: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


In [19]:
from pathlib import Path
import pandas as pd
proteinRMSD_dir = 'output/superimposition/proteinRMSD'

input_dir =  'output/superimposition/superimposed_structures'

df = pd.read_pickle('output/superimposition/superimposedstructures.pkl')

df_proteinRMSD = df << (ProteinRMSD('Entry', input_dir = input_dir, output_dir = proteinRMSD_dir, visualize_heatmaps = True)  
                    >> Save(Path(output_dir)/'proteinRMSD.pkl'))

Processing entry: P53554
output/superimposition/superimposed_structures/P53554/P53554_8_vina__P53554_model_1_boltz.pdb
1.7937351951772424
output/superimposition/superimposed_structures/P53554/P53554_0_chai__P53554_model_0_boltz.pdb
4.83676538468804
output/superimposition/superimposed_structures/P53554/P53554_7_vina__P53554_model_3_boltz.pdb
1.7807677384242657
output/superimposition/superimposed_structures/P53554/P53554_2_vina__P53554_model_0_boltz.pdb
6.481102634216694e-15
output/superimposition/superimposed_structures/P53554/P53554_4_vina__P53554_2_chai.pdb
2.6862192238389095
output/superimposition/superimposed_structures/P53554/P53554_9_vina__P53554_model_1_boltz.pdb
1.7937351951772424
output/superimposition/superimposed_structures/P53554/P53554_7_vina__P53554_model_1_boltz.pdb
1.7937351951772424
output/superimposition/superimposed_structures/P53554/P53554_1_vina__P53554_5_vina.pdb
6.481102634216694e-15
output/superimposition/superimposed_structures/P53554/P53554_4_chai__P53554_0_cha

In [2]:
ligandRMSD_dir = Path('output/superimposition/ligandRMSD')
ligandRMSD_dir.mkdir(exist_ok=True, parents=True) 
input_dir = Path('output/superimposition/superimposed_structures')
df = pd.read_pickle('output/superimposition/proteinRMSD.pkl')
df_best_structures = df << (LigandRMSD('Entry', input_dir = input_dir, output_dir = ligandRMSD_dir, visualize_heatmaps= True, maxMatches = 1000))
df_best_structures_w_metrics = add_metrics_to_best_structures(df_best_structures, pd.read_pickle(Path(output_dir).parent / 'docking/dockingmetrics.pkl'))
df_best_structures_w_metrics.to_pickle(Path(output_dir) / 'best_structures.pkl')
df_best_structures_w_metrics

Processing entry: P53554
Processing entry: P53553


,Entry,tool,best_structure,avg_ligandRMSD,method,ligand_rmsd,boltz_boltz_mean_ligandRMSD,boltz_boltz_std_ligandRMSD,boltz_chai_mean_ligandRMSD,boltz_chai_std_ligandRMSD,...,boltz2_iptm,boltz2_ligand_iptm,boltz2_protein_iptm,boltz2_complex_plddt,boltz2_complex_iplddt,boltz2_complex_pde,boltz2_complex_ipde,boltz2_chains_ptm,boltz2_pair_chains_iptm,vina_affinity
0,P53553,vina,P53553_2_vina,2.353939,inter_tool_weighted_avg,1.305049,2.275021,1.505139,2.794787,1.198758,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,-5.519
1,P53553,vina,P53553_2_vina,2.353939,inter_tool_weighted_avg,4.212166,2.275021,1.505139,2.794787,1.198758,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,-5.519
2,P53553,vina,P53553_2_vina,2.353939,inter_tool_weighted_avg,2.808092,2.275021,1.505139,2.794787,1.198758,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,-5.519
3,P53553,vina,P53553_2_vina,2.353939,inter_tool_weighted_avg,4.892084,2.275021,1.505139,2.794787,1.198758,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,-5.519
4,P53553,vina,P53553_2_vina,2.353939,inter_tool_weighted_avg,1.453972,2.275021,1.505139,2.794787,1.198758,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,-5.519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,P53554,vina,P53554_9_vina,1.286182,vina_avg_intra_tool,13.279149,4.352954,0.902529,7.037866,2.099571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,-2.866
752,P53554,vina,P53554_9_vina,1.286182,vina_avg_intra_tool,13.223626,4.352954,0.902529,7.037866,2.099571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,-2.866
753,P53554,vina,P53554_9_vina,1.286182,vina_avg_intra_tool,6.132955,4.352954,0.902529,7.037866,2.099571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,-2.866
754,P53554,vina,P53554_9_vina,1.286182,vina_avg_intra_tool,4.493215,4.352954,0.902529,7.037866,2.099571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,-2.866


In [3]:
df = pd.read_pickle('output/superimposition/ligandRMSD.pkl')
df

,Entry,pdb_file,docked_structure1,docked_structure2,tool1,tool2,Squidly_CR_Position,proteinRMSD,chai-chai_mean_proteinRMSD,chai-chai_std_proteinRMSD,...,boltz_vina_mean_ligandRMSD,boltz_vina_std_ligandRMSD,chai_chai_mean_ligandRMSD,chai_chai_std_ligandRMSD,chai_vina_mean_ligandRMSD,chai_vina_std_ligandRMSD,vina_vina_mean_ligandRMSD,vina_vina_std_ligandRMSD,overall_ligandRMSD_mean,overall_ligandRMSD_std
0,P53554,P53554_8_vina__P53554_model_1_boltz.pdb,P53554_8_vina,P53554_model_1_boltz,vina,boltz,60|88,1.793735,3.072240,1.160441,...,15.011704,0.643400,3.872761,2.217802,13.377169,0.622233,3.492708,1.244348,10.922145,4.586271
1,P53554,P53554_8_vina__P53554_model_1_boltz.pdb,P53554_8_vina,P53554_model_1_boltz,vina,boltz,60|88,1.793735,3.072240,1.160441,...,15.011704,0.643400,3.872761,2.217802,13.377169,0.622233,3.492708,1.244348,10.922145,4.586271
2,P53554,P53554_8_vina__P53554_model_1_boltz.pdb,P53554_8_vina,P53554_model_1_boltz,vina,boltz,60|88,1.793735,3.072240,1.160441,...,15.011704,0.643400,3.872761,2.217802,13.377169,0.622233,3.492708,1.244348,10.922145,4.586271
3,P53554,P53554_8_vina__P53554_model_1_boltz.pdb,P53554_8_vina,P53554_model_1_boltz,vina,boltz,60|88,1.793735,3.072240,1.160441,...,15.011704,0.643400,3.872761,2.217802,13.377169,0.622233,3.492708,1.244348,10.922145,4.586271
4,P53554,P53554_8_vina__P53554_model_1_boltz.pdb,P53554_8_vina,P53554_model_1_boltz,vina,boltz,60|88,1.793735,3.072240,1.160441,...,15.011704,0.643400,3.872761,2.217802,13.377169,0.622233,3.492708,1.244348,10.922145,4.586271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31747,P53553,P53553_2_vina__P53553_4_chai.pdb,P53553_2_vina,P53553_4_chai,vina,chai,60|88,1.711964,0.574587,0.048463,...,3.491253,1.205118,1.398708,1.153485,3.556555,0.980486,4.013273,1.046104,3.327915,1.261622
31748,P53553,P53553_2_vina__P53553_4_chai.pdb,P53553_2_vina,P53553_4_chai,vina,chai,60|88,1.711964,0.574587,0.048463,...,3.491253,1.205118,1.398708,1.153485,3.556555,0.980486,4.013273,1.046104,3.327915,1.261622
31749,P53553,P53553_2_vina__P53553_4_chai.pdb,P53553_2_vina,P53553_4_chai,vina,chai,60|88,1.711964,0.574587,0.048463,...,3.491253,1.205118,1.398708,1.153485,3.556555,0.980486,4.013273,1.046104,3.327915,1.261622
31750,P53553,P53553_2_vina__P53553_4_chai.pdb,P53553_2_vina,P53553_4_chai,vina,chai,60|88,1.711964,0.574587,0.048463,...,3.491253,1.205118,1.398708,1.153485,3.556555,0.980486,4.013273,1.046104,3.327915,1.261622


### Geometric filtering

In [ ]:
os.mkdir('output/geometricfiltering')
output_dir = ('output/geometricfiltering')

In [ ]:
df_geo_filter = df << (GeometricFiltering(
                                        ligand_smiles=substrate_smiles,
                                        smarts_pattern=smarts_pattern,
                                        preparedfiles_dir=Path(input_dir) / 'preparedfiles_for_superimposition',
                                        output_dir=output_dir,
                                        esterase=0,
                                        find_closest_nucleophile=0
                                    )
                                >> Save(Path(output_dir) / 'geometricfiltering.pkl'))
